# Encoding Operations
This section contains;
* Rare Encoder
* Label Encoder
* One Hot Encoder

## Data Preparation

In [1]:
from pyspark.sql import SparkSession, functions as F
import findspark

findspark.init("C:\Program Files\Spark\spark-3.3.1-bin-hadoop3")

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
spark = SparkSession.builder \
    .appName("House Price Model") \
    .master("local[4]") \
    .config("spark.sql.shuffle.partitions", "2") \
    .getOrCreate()

In [4]:
path = "file:///Users/talha/OneDrive/Masaüstü/Talha Nebi Kumru/Data Science/Miuul/Machine Learning/House_Price_Prediction/datasets/outliers/non_outliers.parquet"
df = spark.read \
    .format("parquet") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load(path)

In [5]:
df.limit(5).toPandas()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60.0,RL,65.0,8450.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7.0,5.0,2003.0,2003.0,Gable,CompShg,VinylSd,VinylSd,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856.0,854.0,0.0,1710.0,1.0,0.0,2.0,1.0,3.0,1.0,Gd,8.0,Typ,0.0,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0.0,61.0,0.0,0.0,0.0,0.0,0.0,2.0,2008.0,WD,Normal,208500.0
1,20.0,RL,80.0,9600.0,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6.0,8.0,1976.0,1976.0,Gable,CompShg,MetalSd,MetalSd,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262.0,0.0,0.0,1262.0,0.0,1.0,2.0,0.0,3.0,1.0,TA,6.0,Typ,1.0,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0,WD,Normal,181500.0
2,60.0,RL,68.0,11250.0,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7.0,5.0,2001.0,2002.0,Gable,CompShg,VinylSd,VinylSd,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920.0,866.0,0.0,1786.0,1.0,0.0,2.0,1.0,3.0,1.0,Gd,6.0,Typ,1.0,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0.0,42.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0,WD,Normal,223500.0
3,70.0,RL,60.0,9550.0,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7.0,5.0,1915.0,1970.0,Gable,CompShg,Wd Sdng,Wd Shng,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961.0,756.0,0.0,1717.0,1.0,0.0,1.0,0.0,3.0,1.0,Gd,7.0,Typ,1.0,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0.0,35.0,272.0,0.0,0.0,0.0,0.0,2.0,2006.0,WD,Abnorml,140000.0
4,60.0,RL,84.0,14260.0,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8.0,5.0,2000.0,2000.0,Gable,CompShg,VinylSd,VinylSd,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145.0,1053.0,0.0,2198.0,1.0,0.0,2.0,1.0,4.0,1.0,Gd,9.0,Typ,1.0,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192.0,84.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0,WD,Normal,250000.0


In [6]:
from _utils._eda_utils import *

In [8]:
numeric_cols, categoric_cols = seperate_columns(df.dtypes)

2023-04-18 09:33:20,996 - _utils.logging_utils - DEBUG - seperate_columns function executing...
2023-04-18 09:33:20,999 - _utils.logging_utils - INFO - 
Numerical Columns: ['Fireplaces', 'PoolArea', 'FullBath', 'GarageCars', 'HalfBath', 'OverallCond', 'MasVnrArea', 'LotFrontage', '1stFlrSF', '3SsnPorch', 'BedroomAbvGr', 'GarageArea', 'WoodDeckSF', 'TotalBsmtSF', 'LowQualFinSF', 'BsmtFinSF1', 'YearBuilt', 'SalePrice', 'BsmtFinSF2', 'BsmtHalfBath', 'GarageYrBlt', 'TotRmsAbvGrd', 'KitchenAbvGr', 'GrLivArea', '2ndFlrSF', 'OpenPorchSF', 'ScreenPorch', 'OverallQual', 'BsmtUnfSF', 'EnclosedPorch', 'YearRemodAdd', 'MoSold', 'YrSold', 'MSSubClass', 'LotArea', 'MiscVal', 'BsmtFullBath']
2023-04-18 09:33:20,999 - _utils.logging_utils - INFO - 
Categorical Columns: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCon

In [9]:
from _utils._sklearn_models import *

In [10]:
def get_score(df):
    X, y = prepare_dataframe(df, 'SalePrice')

    test_model = LinearRegressionSklearn(X, y)
    test_model.init_model()
    
    score = test_model.get_scores()
    
    return score[score[0] == 'r2'][1].values[0] # r2 score

In [13]:
get_score(df.toPandas())

'0.8133674589866953'

## Rare Encoder

### Rare Analyser

In [14]:
from _utils._encoders_utils import *

In [15]:
rare_analyser(df, target='SalePrice', categoric_cols=categoric_cols)

2023-04-18 09:35:07,352 - _utils.logging_utils - DEBUG - rare_analyser function is executing...
+--------+-----+-----+-----------+
|MSZoning|Count|Ratio|Target Mean|
+--------+-----+-----+-----------+
|      RL| 1125| 0.79|   189177.8|
|      RM|  215| 0.15|  126486.83|
|      FV|   61| 0.04|  214492.85|
| C (all)|   10| 0.01|    74528.0|
|      RH|   16| 0.01|  131558.38|
+--------+-----+-----+-----------+

+------+-----+-----+-----------+
|Street|Count|Ratio|Target Mean|
+------+-----+-----+-----------+
|  Pave| 1422|  1.0|  179607.44|
|  Grvl|    5|  0.0|   110438.6|
+------+-----+-----+-----------+

+--------+-----+-----+-----------+
|LotShape|Count|Ratio|Target Mean|
+--------+-----+-----+-----------+
|     IR2|   39| 0.03|  237286.36|
|     Reg|  906| 0.63|  163710.49|
|     IR1|  473| 0.33|  204212.96|
|     IR3|    9| 0.01|  198373.89|
+--------+-----+-----+-----------+

+-----------+-----+-----+-----------+
|LandContour|Count|Ratio|Target Mean|
+-----------+-----+-----+-------

### Rare Encoding

In [16]:
temp_df = df

In [17]:
temp_df = rare_encoder(temp_df, columns=categoric_cols, rare_percent=0.01)

2023-04-18 09:35:37,301 - _utils.logging_utils - DEBUG - rare_analyser function is executing...
2023-04-18 09:35:37,370 - _utils.logging_utils - INFO - MSZoning column is encoding...
2023-04-18 09:35:38,424 - _utils.logging_utils - INFO - Street column is encoding...
2023-04-18 09:35:39,263 - _utils.logging_utils - INFO - LotShape column is encoding...
2023-04-18 09:35:40,142 - _utils.logging_utils - INFO - LandContour column is encoding...
2023-04-18 09:35:40,989 - _utils.logging_utils - INFO - Utilities column is encoding...
2023-04-18 09:35:41,788 - _utils.logging_utils - INFO - LotConfig column is encoding...
2023-04-18 09:35:42,633 - _utils.logging_utils - INFO - LandSlope column is encoding...
2023-04-18 09:35:43,529 - _utils.logging_utils - INFO - Neighborhood column is encoding...
2023-04-18 09:35:44,455 - _utils.logging_utils - INFO - Condition1 column is encoding...
2023-04-18 09:35:45,352 - _utils.logging_utils - INFO - Condition2 column is encoding...
2023-04-18 09:35:46,21

#### Test Rare Encoded

In [19]:
get_score(temp_df.toPandas())

'0.8324483503925876'

In [21]:
df = temp_df

## Label Encoding

In [22]:
binary_cols = [col for col in categoric_cols if df.select(col).distinct().count() == 2]
binary_cols

['Street', 'Utilities', 'Condition2', 'RoofMatl', 'Heating', 'CentralAir']

### Label Encoding

In [24]:
temp_df = df.toPandas()

In [26]:
label_encoder(temp_df, binary_cols)

2023-04-18 09:39:24,262 - _utils.logging_utils - DEBUG - label_encoder function is executing...
2023-04-18 09:39:24,263 - _utils.logging_utils - INFO - Street column is encoding...
2023-04-18 09:39:24,264 - _utils.logging_utils - INFO - Utilities column is encoding...
2023-04-18 09:39:24,266 - _utils.logging_utils - INFO - Condition2 column is encoding...
2023-04-18 09:39:24,267 - _utils.logging_utils - INFO - RoofMatl column is encoding...
2023-04-18 09:39:24,270 - _utils.logging_utils - INFO - Heating column is encoding...
2023-04-18 09:39:24,272 - _utils.logging_utils - INFO - CentralAir column is encoding...


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60.0,RL,65.0,8450.0,0,Reg,Lvl,0,Inside,Gtl,CollgCr,Norm,0,1Fam,2Story,7.0,5.0,2003.0,2003.0,Gable,0,VinylSd,VinylSd,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,0,Ex,1,SBrkr,856.0,854.0,0.0,1710.0,1.0,0.0,2.0,1.0,3.0,1.0,Gd,8.0,Typ,0.0,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0.0,61.0,0.0,0.0,0.0,0.0,0.0,2.0,2008.0,WD,Normal,208500.0
1,20.0,RL,80.0,9600.0,0,Reg,Lvl,0,FR2,Gtl,Rare,Feedr,0,1Fam,1Story,6.0,8.0,1976.0,1976.0,Gable,0,MetalSd,MetalSd,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,0,Ex,1,SBrkr,1262.0,0.0,0.0,1262.0,0.0,1.0,2.0,0.0,3.0,1.0,TA,6.0,Typ,1.0,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0,WD,Normal,181500.0
2,60.0,RL,68.0,11250.0,0,IR1,Lvl,0,Inside,Gtl,CollgCr,Norm,0,1Fam,2Story,7.0,5.0,2001.0,2002.0,Gable,0,VinylSd,VinylSd,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,0,Ex,1,SBrkr,920.0,866.0,0.0,1786.0,1.0,0.0,2.0,1.0,3.0,1.0,Gd,6.0,Typ,1.0,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0.0,42.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0,WD,Normal,223500.0
3,70.0,RL,60.0,9550.0,0,IR1,Lvl,0,Corner,Gtl,Crawfor,Norm,0,1Fam,2Story,7.0,5.0,1915.0,1970.0,Gable,0,Wd Sdng,Wd Shng,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,0,Gd,1,SBrkr,961.0,756.0,0.0,1717.0,1.0,0.0,1.0,0.0,3.0,1.0,Gd,7.0,Typ,1.0,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0.0,35.0,272.0,0.0,0.0,0.0,0.0,2.0,2006.0,WD,Abnorml,140000.0
4,60.0,RL,84.0,14260.0,0,IR1,Lvl,0,FR2,Gtl,NoRidge,Norm,0,1Fam,2Story,8.0,5.0,2000.0,2000.0,Gable,0,VinylSd,VinylSd,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,0,Ex,1,SBrkr,1145.0,1053.0,0.0,2198.0,1.0,0.0,2.0,1.0,4.0,1.0,Gd,9.0,Typ,1.0,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192.0,84.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1422,20.0,FV,62.0,7500.0,0,Reg,Lvl,0,Inside,Gtl,Somerst,Norm,0,1Fam,1Story,7.0,5.0,2004.0,2005.0,Gable,0,VinylSd,VinylSd,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,410.0,Unf,0.0,811.0,1221.0,0,Ex,1,SBrkr,1221.0,0.0,0.0,1221.0,1.0,0.0,2.0,0.0,2.0,1.0,Gd,6.0,Typ,0.0,Attchd,2004.0,RFn,2.0,400.0,TA,TA,Y,0.0,113.0,0.0,0.0,0.0,0.0,0.0,10.0,2009.0,WD,Normal,185000.0
1423,60.0,RL,62.0,7917.0,0,Reg,Lvl,0,Inside,Gtl,Gilbert,Norm,0,1Fam,2Story,6.0,5.0,1999.0,2000.0,Gable,0,VinylSd,VinylSd,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,953.0,953.0,0,Ex,1,SBrkr,953.0,694.0,0.0,1647.0,0.0,0.0,2.0,1.0,3.0,1.0,TA,7.0,Typ,1.0,Attchd,1999.0,RFn,2.0,460.0,TA,TA,Y,0.0,40.0,0.0,0.0,0.0,0.0,0.0,8.0,2007.0,WD,Normal,175000.0
1424,20.0,RL,85.0,13175.0,0,Reg,Lvl,0,Inside,Gtl,NWAmes,Norm,0,1Fam,1Story,6.0,6.0,1978.0,1988.0,Gable,0,Plywood,Plywood,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790.0,Rec,163.0,589.0,1542.0,0,TA,1,SBrkr,2073.0,0.0,0.0,2073.0,1.0,0.0,2.0,0.0,3.0,1.0,TA,7.0,Min1,2.0,Attchd,1978.0,Unf,2.0,500.0,TA,TA,Y,349.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2010.0,WD,Normal,210000.0
1425,20.0,RL,68.0,9717.0,0,Reg,Lvl,0,Inside,Gtl,NAmes,Norm,0,1Fam,1Story,5.0,6.0,1950.0,1996.0,Hip,0,MetalSd,MetalSd,0.0,TA,TA,CB

In [27]:
temp_df[binary_cols].head()

,Street,Utilities,Condition2,RoofMatl,Heating,CentralAir
0,0,0,0,0,0,1
1,0,0,0,0,0,1
2,0,0,0,0,0,1
3,0,0,0,0,0,1
4,0,0,0,0,0,1


In [31]:
df = temp_df

In [32]:
df.to_parquet("datasets/encode/encoded.parquet", index=False)

In [33]:
spark.stop()

## Conclusion

<div class="alert alert-block alert-info"> <b>Final:</b> Final encoding, accuracy increased <b>0.8133<b> to <b>0.8324<b> </div>